<a href="https://colab.research.google.com/github/bernbr/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing - Attempt 1

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts<500].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification = class_counts[class_counts>1]
classification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts<1000].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model - Attempt 1

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=130, activation="tanh", input_dim=43))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=130, activation="relu"))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units=100, activation="tanh"))

# Fourth hidden layer
nn1.add(tf.keras.layers.Dense(units=100, activation="relu"))

# Fifth hidden layer
nn1.add(tf.keras.layers.Dense(units=130, activation="tanh"))

# Sixth hidden layer
nn1.add(tf.keras.layers.Dense(units=130, activation="relu"))

# Seventh hidden layer
nn1.add(tf.keras.layers.Dense(units=40, activation="tanh"))

# Sixth hidden layer
nn1.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 130)               5720      
                                                                 
 dense_1 (Dense)             (None, 130)               17030     
                                                                 
 dense_2 (Dense)             (None, 100)               13100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 130)               13130     
                                                                 
 dense_5 (Dense)             (None, 130)               17030     
                                                                 
 dense_6 (Dense)             (None, 40)                5

In [ ]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
y_test = np.array(y_test)
y_train = np.array(y_train)

In [ ]:
# Create a callback that saves the model's weights every five epochs
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoints variables
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path1 = 'checkpoints/weights.{epoch}.hdf5'

# Create callback 
callback1 = ModelCheckpoint(filepath=checkpoint_path1,
                              verbose=1,
                              save_weights_only=True,
                              save_freq='epoch',
                              period=5)

In [ ]:
# Train the model 
fit_model = nn1.fit(X_train_scaled, y_train, epochs=100, callbacks=[callback1])

Epoch 1/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5489 - accuracy: 0.7317
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5502 - accuracy: 0.7335
Epoch 3/100
802/804 [============================>.] - ETA: 0s - loss: 0.5520 - accuracy: 0.7360
Epoch 3: saving model to checkpoints/weights.3.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5520 - accuracy: 0.7360
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5507 - accuracy: 0.7348
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5492 - accuracy: 0.7367
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5502 - accuracy: 0.7352
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5533 - accuracy: 0.7361
Epoch 8/100
796/804 [============================>.] - ETA: 0s - loss: 0.5562 - accuracy: 0.7365
Epoch 8: saving model to checkpoints/weights.8.hdf5
804/804

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5687 - accuracy: 0.7205 - 689ms/epoch - 3ms/step
Loss: 0.5687193274497986, Accuracy: 0.7204664945602417


In [ ]:
# Export our model to HDF5 file
from tensorflow import keras
from keras.models import load_model
model_1 = nn1.save('AlphabetSoupCharity_Optimization1.h5')
load_model_1 = keras.models.load_model("AlphabetSoupCharity_Optimization1.h5")
print(load_model_1)

In [ ]:
# Download from Colab to H5 file
from google.colab import files
files.download("AlphabetSoupCharity_Optimization1.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Compile, Train and Evaluate the Model - Attempt 2

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=150, activation="sigmoid", input_dim=43))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=150, activation="tanh"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=130, activation="tanh"))

# Fourth hidden layer
nn2.add(tf.keras.layers.Dense(units=130, activation="tanh"))

# Fifth hidden layer
nn2.add(tf.keras.layers.Dense(units=100, activation="tanh"))

# Sixth hidden layer
nn2.add(tf.keras.layers.Dense(units=100, activation="tanh"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 150)               6600      
                                                                 
 dense_10 (Dense)            (None, 150)               22650     
                                                                 
 dense_11 (Dense)            (None, 130)               19630     
                                                                 
 dense_12 (Dense)            (None, 130)               17030     
                                                                 
 dense_13 (Dense)            (None, 100)               13100     
                                                                 
 dense_14 (Dense)            (None, 100)               10100     
                                                                 
 dense_15 (Dense)            (None, 1)                

In [ ]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
y_test = np.array(y_test)
y_train = np.array(y_train)

In [ ]:
# Create a callback that saves the model's weights every five epochs
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoints variables
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path2 = 'checkpoints/weights.{epoch}.hdf5'

# Create callback 
callback2 = ModelCheckpoint(filepath=checkpoint_path2,
                              verbose=1,
                              save_weights_only=True,
                              save_freq='epoch',
                              period=5)

In [ ]:
# Train the model 
fit_model = nn2.fit(X_train_scaled, y_train, epochs=50, callbacks=[callback2])

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.5880 - accuracy: 0.7088
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5694 - accuracy: 0.7230
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5701 - accuracy: 0.7250
Epoch 4/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5659 - accuracy: 0.7256
Epoch 5/50
802/804 [============================>.] - ETA: 0s - loss: 0.5639 - accuracy: 0.7238
Epoch 5: saving model to checkpoints/weights.5.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5640 - accuracy: 0.7238
Epoch 6/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5612 - accuracy: 0.7271
Epoch 7/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5620 - accuracy: 0.7255
Epoch 8/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5599 - accuracy: 0.7276
Epoch 9/50
804/804 [==============================] - 3s 4ms/st

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5527 - accuracy: 0.7305 - 613ms/epoch - 2ms/step
Loss: 0.5526911020278931, Accuracy: 0.7304956316947937


In [ ]:
# Export our model to HDF5 file
from tensorflow import keras
from keras.models import load_model
model_2 = nn2.save('AlphabetSoupCharity_Optimization2.h5')
load_model_2 = keras.models.load_model("AlphabetSoupCharity_Optimization2.h5")

In [ ]:
# Download from Colab to H5 file
from google.colab import files
files.download("AlphabetSoupCharity_Optimization2.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Compile, Train and Evaluate the Model - Attempt 3

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=100, activation="sigmoid", input_dim=43))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=90, activation="sigmoid"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=60, activation="sigmoid"))

# Forth hidden layer
nn3.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 100)               4400      
                                                                 
 dense_47 (Dense)            (None, 90)                9090      
                                                                 
 dense_48 (Dense)            (None, 60)                5460      
                                                                 
 dense_49 (Dense)            (None, 20)                1220      
                                                                 
 dense_50 (Dense)            (None, 1)                 21        
                                                                 
Total params: 20,191
Trainable params: 20,191
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
y_test = np.array(y_test)
y_train = np.array(y_train)

In [ ]:
# Create a callback that saves the model's weights every five epochs
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoints variables
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path= 'checkpoints/weights.{epoch}.hdf5'

# Create callback 
callback3 = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq='epoch',
                              period=5)

In [ ]:
# Train the model 
fit_model = nn3.fit(X_train_scaled, y_train, epochs=40, callbacks=[callback3])

Epoch 1/40
804/804 [==============================] - 3s 3ms/step - loss: 0.5966 - accuracy: 0.7006
Epoch 2/40
804/804 [==============================] - 2s 3ms/step - loss: 0.5745 - accuracy: 0.7207
Epoch 3/40
804/804 [==============================] - 3s 4ms/step - loss: 0.5665 - accuracy: 0.7241
Epoch 4/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5623 - accuracy: 0.7253
Epoch 5/40
788/804 [============================>.] - ETA: 0s - loss: 0.5614 - accuracy: 0.7266
Epoch 5: saving model to checkpoints/weights.5.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5609 - accuracy: 0.7270
Epoch 6/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5588 - accuracy: 0.7274
Epoch 7/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7289
Epoch 8/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 - accuracy: 0.7289
Epoch 9/40
804/804 [==============================] - 3s 4ms/st

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5497 - accuracy: 0.7332 - 517ms/epoch - 2ms/step
Loss: 0.5496671199798584, Accuracy: 0.7331778407096863


In [ ]:
# Export our model to HDF5 file
from tensorflow import keras
from keras.models import load_model
model_3 = nn3.save('AlphabetSoupCharity_Optimization3.h5')
load_model_3 = keras.models.load_model("AlphabetSoupCharity_Optimization3.h5")

In [ ]:
# Download from Colab to H5 file
from google.colab import files
files.download("AlphabetSoupCharity_Optimization1.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>